# 🥋 Lekcja 49: Custom Loss Functions (Triplet Loss & Vectorization)

Pisanie własnej funkcji kosztu w PyTorch jest proste: wystarczy napisać funkcję, która przyjmuje Tensory i zwraca skalar, używając operacji różniczkowalnych PyTorcha.

Trudność leży w **wydajności** i **stabilności numerycznej**.

**Studium przypadku: Triplet Loss**
Chcemy nauczyć sieć, że:
*   Twarz A (Anchor) jest podobna do Twarzy P (Positive).
*   Twarz A jest różna od Twarzy N (Negative).

Wzór:
$$ L = \max(0, \text{dist}(A, P) - \text{dist}(A, N) + \text{margin}) $$

Wyzwaniem jest obliczenie odległości euklidesowej dla całego batcha naraz, bez pętli.

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Konfiguracja
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

print(f"Urządzenie: {DEVICE}")

Urządzenie: cuda


## Wersja 1: Naiwna (Powolna)

Zaimplementujmy to "po ludzku", używając wbudowanej funkcji `pairwise_distance`.
To działa, ale w bardziej skomplikowanych wariantach (np. szukanie najtrudniejszych negatywów w batchu - Hard Mining) wymagałoby pętli.

In [2]:
class NaiveTripletLoss(nn.Module):
    def __init__(self, margin=1.0):
        super().__init__()
        self.margin = margin
        
    def forward(self, anchor, positive, negative):
        # anchor, positive, negative: [Batch, Embed_Dim]
        
        # 1. Liczymy dystanse
        dist_pos = F.pairwise_distance(anchor, positive, p=2)
        dist_neg = F.pairwise_distance(anchor, negative, p=2)
        
        # 2. Wzór Hinge Loss
        loss = torch.relu(dist_pos - dist_neg + self.margin)
        
        return loss.mean()

# Test
criterion_naive = NaiveTripletLoss()
a = torch.randn(32, 128, requires_grad=True).to(DEVICE)
p = torch.randn(32, 128, requires_grad=True).to(DEVICE)
n = torch.randn(32, 128, requires_grad=True).to(DEVICE)

loss = criterion_naive(a, p, n)
print(f"Naive Loss: {loss.item():.4f}")

Naive Loss: 1.4534


## Wersja 2: Professional (Macierzowa)

W zaawansowanych systemach (np. SimCLR, Metric Learning) często musimy policzyć macierz odległości **każdy z każdym** wewnątrz batcha.
Użycie pętli jest tu zabójcze.

Użyjemy wzoru skróconego mnożenia dla odległości euklidesowej:
$$ ||A - B||^2 = ||A||^2 + ||B||^2 - 2 \cdot A \cdot B^T $$

Dzięki temu możemy użyć ultraszybkiego mnożenia macierzy (`@` lub `matmul`).

**Pułapka NaN:**
Pochodna z $\sqrt{x}$ to $\frac{1}{2\sqrt{x}}$.
Jeśli $x=0$ (dystans wynosi zero, bo obrazy są identyczne), mianownik wynosi 0 -> Gradient wybucha do `inf` -> Wagi stają się `NaN`.
Musimy dodać mały $\epsilon$ przed pierwiastkowaniem.

In [3]:
def pairwise_distance_matrix(x, y):
    """
    Oblicza dystans Euklidesowy między każdym elementem x a każdym elementem y.
    x: [N, D]
    y: [M, D]
    Wynik: [N, M]
    """
    # 1. Kwadraty norm
    x_sq = torch.sum(x**2, dim=1, keepdim=True) # [N, 1]
    y_sq = torch.sum(y**2, dim=1, keepdim=True) # [M, 1] -> transponujemy wirtualnie do [1, M]
    
    # 2. Iloczyn skalarny (2ab)
    # [N, D] @ [D, M] -> [N, M]
    prod = torch.matmul(x, y.t())
    
    # 3. Wzór (a^2 + b^2 - 2ab)
    # Broadcasting zadba o wymiary: [N, 1] + [1, M] - [N, M] -> [N, M]
    dist_sq = x_sq + y_sq.t() - 2 * prod
    
    # 4. Zabezpieczenie przed ujemnymi zerami (błędy float)
    dist_sq = torch.clamp(dist_sq, min=1e-12)
    
    return torch.sqrt(dist_sq)

class AdvancedTripletLoss(nn.Module):
    def __init__(self, margin=1.0):
        super().__init__()
        self.margin = margin
        
    def forward(self, anchor, positive, negative):
        # Tutaj liczymy tylko pary (i, i), ale dzięki funkcji macierzowej
        # moglibyśmy łatwo zaimplementować "Batch Hard Mining" (najtrudniejszy negatyw w całym batchu).
        
        # Obliczamy dystanse
        # Uwaga: funkcja zwraca macierz NxN, my chcemy tylko przekątną (odległość pary i-i)
        # Ale dla edukacji użyjemy tej funkcji.
        
        # Dystans A-P
        dists_ap = pairwise_distance_matrix(anchor, positive)
        # Bierzemy przekątną (dystans między anchor[i] a positive[i])
        d_ap = torch.diag(dists_ap)
        
        # Dystans A-N
        dists_an = pairwise_distance_matrix(anchor, negative)
        d_an = torch.diag(dists_an)
        
        loss = torch.relu(d_ap - d_an + self.margin)
        return loss.mean()

print("Zaawansowana funkcja kosztu gotowa.")

Zaawansowana funkcja kosztu gotowa.


## Weryfikacja: Gradienty i NaN

Sprawdźmy, czy nasza funkcja jest stabilna.
Stworzymy przypadek, gdzie `anchor == positive` (dystans = 0).
W naiwnej implementacji (bez epsilora) `backward()` mógłby zwrócić `NaN`.

In [5]:
criterion_adv = AdvancedTripletLoss()

# --- POPRAWKA ---
# Tworzymy tensor BEZPOŚREDNIO na urządzeniu (device=DEVICE).
# Dzięki temu 'a_zero' jest Liściem (Leaf Tensor) i jego .grad zostanie zachowany.
a_zero = torch.randn(5, 10, device=DEVICE, requires_grad=True)

# p_zero to klon a_zero.
# Uwaga: p_zero nie jest liściem (jest wynikiem klonowania), 
# ale nas interesuje gradient na 'a_zero', więc jest OK.
p_zero = a_zero.clone() 

n_zero = torch.randn(5, 10, device=DEVICE, requires_grad=True)

# Liczymy stratę
loss = criterion_adv(a_zero, p_zero, n_zero)

print(f"Loss przy idealnym dopasowaniu: {loss.item()}")

# Próba Backward
try:
    loss.backward()
    
    # Teraz a_zero.grad będzie istniał i nie będzie ostrzeżenia
    grad_norm = a_zero.grad.norm().item()
    print(f"Gradient Anchora (norma): {grad_norm}")
    
    if torch.isnan(a_zero.grad).any():
        print("❌ BŁĄD: Gradient to NaN! (Dzielenie przez zero w pierwiastku)")
    else:
        print("✅ SUKCES: Gradient jest stabilny (dzięki clamp/epsilon).")
        
except Exception as e:
    print(f"Błąd: {e}")

Loss przy idealnym dopasowaniu: 0.0
Gradient Anchora (norma): 0.0
✅ SUKCES: Gradient jest stabilny (dzięki clamp/epsilon).


## 🥋 Black Belt Summary

1.  **Unikaj pętli `for`** w funkcjach kosztu. Jeśli masz batcha, używaj operacji macierzowych (`matmul`, broadcasting).
2.  **`clamp(min=1e-8)`**: Zawsze używaj tego przed pierwiastkowaniem (`sqrt`) lub logarytmowaniem (`log`). W Deep Learningu zero jest Twoim wrogiem przy liczeniu pochodnych.
3.  **Wzór skróconego mnożenia:** $||a-b||^2 = a^2 + b^2 - 2ab$ to najszybszy sposób na policzenie macierzy odległości na GPU.